In [ ]:
#|default_exp net.test_graph_mermaid

In [ ]:
#|export
import fbdev

In [ ]:
#|export
from typing import Type, Optional, Union, Any, Tuple, Dict
import asyncio

from fbdev.component import func_component, PortSpecCollection, PortTypeSpec, PortSpec, PortType, DummyComponentFactory
from fbdev.graph import Graph, NodeSpec, EdgeSpec
from fbdev.node import Node, GraphComponentFactory
from fbdev.execute import BatchExecutor

In [ ]:
#|export
sub_graph = Graph(PortSpecCollection(
    input=PortTypeSpec(subgraph_in=PortSpec(dtype=int)),
    output=PortTypeSpec(subgraph_out=PortSpec(dtype=int))
))
sub_graph_component = GraphComponentFactory.get_component(sub_graph)
sub_graph_node = NodeSpec(sub_graph_component)
dummy_component = DummyComponentFactory.get_component(
    inputs=['in'],
    outputs=['out'],
    component_name='dummy'
)

sub_graph.add_edge(EdgeSpec())
sub_graph.add_edge(EdgeSpec())
sub_graph.add_node(NodeSpec(dummy_component))

sub_graph.connect_edge_to_graph_port(PortType.INPUT, 'subgraph_in', 0)
sub_graph.connect_edge_to_node('dummy', PortType.INPUT, 'in', 0)
sub_graph.connect_edge_to_graph_port(PortType.OUTPUT, 'subgraph_out', 1)
sub_graph.connect_edge_to_node('dummy', PortType.OUTPUT, 'out', 1)

top_graph = Graph(PortSpecCollection(
    input=PortTypeSpec(graph_in=PortSpec(dtype=int)),
    output=PortTypeSpec(graph_out=PortSpec(dtype=int))
))

top_graph.add_edge(EdgeSpec())
top_graph.add_edge(EdgeSpec())
top_graph.add_node(NodeSpec(sub_graph_component), id='subgraph')

top_graph.connect_edge_to_graph_port(PortType.INPUT, 'graph_in', 0)
top_graph.connect_edge_to_node('subgraph', PortType.INPUT, 'subgraph_in', 0)
top_graph.connect_edge_to_graph_port(PortType.OUTPUT, 'graph_out', 1)
top_graph.connect_edge_to_node('subgraph', PortType.OUTPUT, 'subgraph_out', 1)

In [ ]:
sub_graph.display_mermaid()

```mermaid
flowchart 
    classDef subgraph_zone fill:#111
    classDef loose_edge fill:#f00

    subgraph NET__CHILD__dummy["Node(dummy)[]"]
        NET__CHILD__dummy__PORTS__input__CHILD__in{{in}}
        NET__CHILD__dummy__PORTS__output__CHILD__out([out])
    end

        NET__PORTS__input__CHILD__subgraph_in{{subgraph_in}}
        NET__PORTS__output__CHILD__subgraph_out([subgraph_out])

    NET__PORTS__input__CHILD__subgraph_in -.-> NET__CHILD__dummy__PORTS__input__CHILD__in
    NET__CHILD__dummy__PORTS__output__CHILD__out -.-> NET__PORTS__output__CHILD__subgraph_out

```

In [ ]:
top_graph.display_mermaid()

```mermaid
flowchart 
    classDef subgraph_zone fill:#111
    classDef loose_edge fill:#f00

    subgraph NET__CHILD__subgraph["Node(GraphComponent)[subgraph]"]
        subgraph NET__CHILD__subgraph__PROP__children[" "]
            subgraph NET__CHILD__subgraph__CHILD__dummy["Node(dummy)[]"]
                NET__CHILD__subgraph__CHILD__dummy__PORTS__input__CHILD__in{{in}}
                NET__CHILD__subgraph__CHILD__dummy__PORTS__output__CHILD__out([out])
            end
        end
            NET__CHILD__subgraph__PORTS__input__CHILD__subgraph_in{{subgraph_in}}
            NET__CHILD__subgraph__PORTS__output__CHILD__subgraph_out([subgraph_out])
    end

        NET__PORTS__input__CHILD__graph_in{{graph_in}}
        NET__PORTS__output__CHILD__graph_out([graph_out])

    NET__CHILD__subgraph__PORTS__input__CHILD__subgraph_in -.-> NET__CHILD__subgraph__CHILD__dummy__PORTS__input__CHILD__in
    NET__CHILD__subgraph__CHILD__dummy__PORTS__output__CHILD__out -.-> NET__CHILD__subgraph__PORTS__output__CHILD__subgraph_out
    NET__PORTS__input__CHILD__graph_in -.-> NET__CHILD__subgraph__PORTS__input__CHILD__subgraph_in
    NET__CHILD__subgraph__PORTS__output__CHILD__subgraph_out -.-> NET__PORTS__output__CHILD__graph_out

    class NET__CHILD__subgraph__PROP__children subgraph_zone
```

In [ ]:
#|export
def test_graph_to_mermaid():
    sub_graph.to_mermaid()
    top_graph.to_mermaid()